# AD9833 Modulators test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'FX2LP', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline 

from signal_generators import modulators  
from clock_generators.si5351 import Si5351
from utilities.adapters import peripherals
import fx2lp

Populating the interactive namespace from numpy and matplotlib
No USB device matches URL ftdi://ftdi:ft232h/1


## Debug mode?

In [5]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = False

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [7]:
bus = fx2lp.I2C(as_400KHz = True)

In [8]:
si = cls(bus) 
si2 = cls(bus)   

In [32]:
freq_demo = 3.05e7

bfsk = modulators.BFSK(si)
bpsk = modulators.BPSK(si, freq = freq_demo)
dtmf = modulators.DTMF((si, si2))
fm = modulators.FM(si)
iq = modulators.IQ((si, si2), freq = freq_demo)
ook = modulators.OOK(si)
pm = modulators.PM(si, freq = freq_demo)
pwm = modulators.PWM(si)
qpsk = modulators.QPSK(si, freq = freq_demo)

## Testing data sequence

In [33]:
samples_size = 10
duration = 0.1

symbols = np.random.randint(2, size = samples_size)
digital_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.uniform(-1, 1, size = samples_size).round(5)
analog_sequence = [(float(symbol), duration) for symbol in symbols] 


symbols = np.random.choice(['1', '2', '3', 'A', '4', '5', '6', 'B', '7', '8', '9', 'C', '*', '0', '#', 'D'], 
                           size = samples_size)
dtmf_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice([0, 1, 2, 3], size = samples_size)
quadrature_sequence = [(symbol, duration) for symbol in symbols] 

## Modulators test

In [34]:
def dump(m):
    m._device.dump()

In [35]:
# bfsk.send_sequence(digital_sequence)

In [36]:
# bpsk.send_sequence(digital_sequence)

In [37]:
ook.send_sequence(digital_sequence)

In [38]:
fm.send_sequence(analog_sequence)

In [39]:
pm.send_sequence(analog_sequence)

In [40]:
qpsk.send_sequence(quadrature_sequence)

In [41]:
# iq.send_sequence(quadrature_sequence)

In [42]:
dtmf.FREQUENCIES = {'row'   : (6970000, 7700000, 8520000, 9410000),
                   'column': (12090000, 13360000, 14770000, 16330000)}

dtmf.TONES = {'1': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][0]),
             '2': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][1]),
             '3': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][2]),
             'A': (dtmf.FREQUENCIES['row'][0], dtmf.FREQUENCIES['column'][3]),
             '4': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][0]),
             '5': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][1]),
             '6': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][2]),
             'B': (dtmf.FREQUENCIES['row'][1], dtmf.FREQUENCIES['column'][3]),
             '7': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][0]),
             '8': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][1]),
             '9': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][2]),
             'C': (dtmf.FREQUENCIES['row'][2], dtmf.FREQUENCIES['column'][3]),
             '*': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][0]),
             '0': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][1]),
             '#': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][2]),
             'D': (dtmf.FREQUENCIES['row'][3], dtmf.FREQUENCIES['column'][3])}
dtmf.SYMBOLS = list(dtmf.TONES.keys())

In [43]:
dtmf.send_sequence(dtmf_sequence)

In [44]:
dtmf._devices[1].clocks[0].set_frequency(25e6)

True

In [45]:
dtmf._devices[1].clocks[0].enable()

In [46]:
dtmf._devices[0].clocks[0].status

OrderedDict([('type', '_Clock'),
             ('idx', 0),
             ('source_type', '_Multisynth'),
             ('source_idx', 0),
             ('source_freq', 7700000),
             ('my_freq', 7700000),
             ('my_divider', 1),
             ('power_downed', False),
             ('enabled', False),
             ('oeb_pin_masked', False),
             ('phase_offset_enabled', False),
             ('phase', 0)])

In [47]:
dtmf._devices[0].clocks[0].freq

7700000

In [48]:
pwm.run()

User interrupts.
